In [3]:
import cv2
import tkinter as tk
import tkinter.filedialog as tkfd
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import  Flatten, Dense, Activation,Convolution2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K
import random
%matplotlib inline

In [4]:
def load_images(folder):
        image_list=[]
        for filename in os.listdir(folder):
            img=cv2.imread(os.path.join(folder,filename))
            if img is not None:
                image_list.append(img)
        return image_list

In [5]:
X_Train=load_images("Image-Data\Training Images")
X_Train=np.array(X_Train)

In [6]:
X_Test=load_images("Image-Data\Testing Images")
X_Test=np.array(X_Test)

In [7]:
X_Validation=load_images("Image-Data\Validation Images")
X_Validation=np.array(X_Validation)

In [8]:
data_Train=pd.read_csv("ISIC-2017_Training_Part3_GroundTruth.csv")
data_Test=pd.read_csv("ISIC-2017_Test_v2_Part3_GroundTruth.csv")
data_Validation=pd.read_csv("ISIC-2017_Validation_Part3_GroundTruth.csv")

In [9]:
data_Train=data_Train.iloc[0:2000,1]
data_Test=data_Test.iloc[0:600,1]
data_Validation=data_Validation.iloc[0:300,1]

y_Train=data_Train
y_Test=data_Test
y_Validation=data_Validation

y_Train=np.array(y_Train)
y_Test=np.array(y_Test)
y_Validation=np.array(y_Validation)

In [10]:
y_Train = np_utils.to_categorical(data_Train)
y_Test = np_utils.to_categorical(data_Test)
y_Validation = np_utils.to_categorical(data_Validation)

In [17]:
# Convolutional Neural Network with 6 layers
Optimizer=Adam(learning_rate=0.001)
objective='binary_crossentropy'
def center_normalize(x):
    return (x-K.mean(x))/K.std(x)
model=Sequential()
#input layer
model.add(Activation(activation=center_normalize, input_shape=(64, 64,3)))
# convolutional layer
model.add(Convolution2D(32,5,5,padding='same',activation='relu',data_format='channels_last'))
#pooling layer
model.add(MaxPooling2D(pool_size=(2,2),data_format='channels_last'))
# convolutional layer
model.add(Convolution2D(64,3,3,padding='same',activation='relu',data_format='channels_last'))
# pooling layer
model.add(MaxPooling2D(pool_size=(2,2),data_format='channels_last'))
model.add(Flatten())
# Relu 
model.add(Dense(128, activation='relu'))
# Sigmoid Fully connected layer
model.add(Dense(y_Train.shape[1]))
model.add(Activation('sigmoid'))
model.compile(loss=objective,optimizer=Optimizer,metrics=['accuracy'])

In [19]:
random.seed(250)
model.fit(X_Train,y_Train,batch_size=200,epochs=15,verbose=1)

Epoch 1/15
10/10 [==============================] - 7s 10ms/step - loss: 0.5664 - accuracy: 0.7465
Epoch 2/15
10/10 [==============================] - 0s 8ms/step - loss: 0.5154 - accuracy: 0.8130
Epoch 3/15
10/10 [==============================] - 0s 7ms/step - loss: 0.5018 - accuracy: 0.8130
Epoch 4/15
10/10 [==============================] - 0s 7ms/step - loss: 0.4903 - accuracy: 0.8130
Epoch 5/15
10/10 [==============================] - 0s 7ms/step - loss: 0.4824 - accuracy: 0.8130
Epoch 6/15
10/10 [==============================] - 0s 7ms/step - loss: 0.4739 - accuracy: 0.8130
Epoch 7/15
10/10 [==============================] - 0s 7ms/step - loss: 0.4636 - accuracy: 0.8130
Epoch 8/15
10/10 [==============================] - 0s 8ms/step - loss: 0.4620 - accuracy: 0.8130
Epoch 9/15
10/10 [==============================] - 0s 7ms/step - loss: 0.4557 - accuracy: 0.8135
Epoch 10/15
10/10 [==============================] - 0s 8ms/step - loss: 0.4483 - accuracy: 0.8155
Epoch 11/15
10/10 

In [20]:
scores = model.evaluate(X_Test, y_Test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

19/19 [==============================] - 1s 6ms/step - loss: 0.5213 - accuracy: 0.8050

accuracy: 80.50%


In [21]:
scores = model.evaluate(X_Validation, y_Validation)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

5/5 [==============================] - 0s 9ms/step - loss: 0.5032 - accuracy: 0.8000

accuracy: 80.00%
